# About Synchronizing LoRa Frames

## 1. Qué significa sincronizar en LoRa

La sincronización de una trama LoRa implica alinear un buffer de muestras IQ tal que la muestra 0 coincida con el comienzo del payload. Es decir, el sistema debe buscar la ubicación de una trama válida dentro del buffer, determinar su estructura (preamble, sync word, SFD, header y payload), y recortar el buffer de entrada a partir del inicio útil de la trama.

El objetivo es eliminar cualquier ruido anterior al inicio real de la señal de interés, facilitando el proceso de decodificación posterior.


## 2. Métodos implementados

Este trabajo implementa dos métodos de sincronización para detectar el inicio de trama:

**2.1 Dechirping-Based Synchronization**  
Una técnica más protocolar que se basa en demodular secciones del buffer de manera sistemática para detectar estructuras características del protocolo LoRa, como corridas de símbolos repetidos (preamble) y pares específicos (como el doble downchirp del SFD). A partir de eso, se deduce la ubicación de la trama.

**2.2 Correlation-Based Synchronization**  
Un método clásico que compara segmentos del buffer IQ con chirps ideales usando correlación cruzada.


## 3. Sincronización por Dechirping

En este trabajo y específicmamente para este sincronizador separamos la sincronización en tres niveles de alineamiento, cada uno asociado a una granularidad distinta y a una porción específica del código:

**Alineación de a Muestra**  
Corresponde a desalineaciones de una o pocas muestras (menores a un chip). No cambian el símbolo recibido, pero sí degradan la energía detectada al no estar perfectamente centrados sobre el pico del chirp. Esto se puede corregir a partir de la función `generate_subchip_view`, que construye múltiples vistas del buffer IQ con distintos desplazamientos de muestra.

La correción como tal sucede en la sección de correción con ventana de desplazamiento.

```
sub_chip_offset = row_idx
```

**Alineación de a Chip**  
Corresponde a desalineaciones de múltiplos de un chip dentro de un símbolo. Sí afectan el símbolo decodificado (puede resultar en símbolos vecinos) pero no a la potencia, su desalineación sería con respecto a los bins de la FFT al demodular.

Por ejemplo, donde hay upchirps (y por ende ceros) se podría llegar a demodular el símbolo 1 o $2^(SF)-1$ si la señal está desalineada temporalmente en un chip.

Se representa con el campo `symbol` del candidato encontrado, que indica cuántos chips corridos está el peak dentro del símbolo.  
Se corrige en:

```python
chip_offset = cand.symbol * samples_per_chip
```
**Alineación de a Símbolo**  
Corresponde a desplazamientos enteros de símbolo, es decir, en qué símbolo dentro de la señal ocurre la corrida candidata del preámbulo.
Se representa con el campo `col_idx` dle candidato encontrado, que indica el índice del primer símbolo detectado en la corrida.  
Se corrige en:

```python
symbol_offset = cand.col_idx * samples_per_symbol
```

**Offset total de sincronización:**  
La suma de los tres offsets permite alinear el buffer IQ al comienzo estimado de la trama:

```python
sync_offset = sub_chip_offset + chip_offset + symbol_offset
synced = iq_samples[sync_offset:]
```

A partir de `synced`, el resto del proceso busca confirmar si efectivamente hay una trama válida (buscando el SFD, decodificando el header, etc.). En las subsecciones siguientes se detalla en más profundidad como es que se estiman estos desplazamientos y como se termina devolviendo la carga útil lista para demodular.

### 3.1 Construcción de Vistas Desplazadas del Buffer (sub-chip alignment)

Se construyen múltiples vistas desfasadas de a una muestra del buffer IQ para capturar alineamientos sub-chip. Cada fila representa una posible desalineación y se evaluará más adelante.

> **NOTA** Se construyen `LoRaPhyParams.samples_per_chip` vistas, es decir, TODAS las vistas posibles y por lo tanto contempla toda sincronización sub-chip que se pueda concluir.

### 3.2 Demodulación exploratoria

Se aplica demodulación por `dechirping` a cada vista del buffer para obtener símbolos (`up_syms`) y sus magnitudes (`up_mags`). Estas observaciones alimentan al proceso de detección de candidatos. 

Esto se puede realizar gracias a que el demodulador acepta buffers bidimensionales, ya que tuvo que ser refactorizado para poder realizar esto de manera eficiente y vectorizada.

### 3.3 Detección de candidatos de sincronización

Se buscan runs (corridas) largas de símbolos repetidos. Usualmente, esto indica que estamos ante los marcadores llamados upchirp, que no son más que símbolos "cero" pero ubicados en el preámbulo $^1$. Aquellas corridas que cumplan con ciertas condiciones (longitud dentro de un rango razonable) se consideran candidatas a ser el preámbulo.

```python
candidates = best_repeated_runs(...)
```

Cada candidato lleva consigo los tres offsets antes mencionados (row_idx, symbol, col_idx) y un score que se calcula en base a dos factores:

- $ \text{run\_score} $: magnitud promedio de la corrida  
- $ \text{run\_len\_score} $: qué tan cerca está la longitud de la corrida del valor ideal

Este score total se lo calcula como:

$$
\text{run\_score}_{\text{norm}} = \frac{\text{run\_score} - \min(\text{run\_scores})}{\max(\text{run\_scores}) - \min(\text{run\_scores}) + \varepsilon}
$$

$$
\text{run\_len\_score} = \max\left(0,\ 1 - \frac{|\text{run\_len} - \text{ideal\_upchirps}|}{\text{ideal\_upchirps}}\right)
$$

$$
\text{total\_score} = 0.7 \cdot \text{run\_score}_{\text{norm}} + 0.3 \cdot \text{run\_len\_score}
$$

Por último, los candidatos se ordenan por este $ \text{total\_score} $ y se seleccionan los mejores.


> $^1$ No siempre serán ceros debido al sub-symbol disalignment, de hecho, casi nunca lo son y justamente eso nos permite alinear la trama a nivel de chips: Como deberían ser ceros y no lo son, corremos al IQ buffer la cantidad de chips necesairos para que lo sean.

### 3.4 Evaluación de candidatos

Luego de ordenar los candidatos por su `total_score`, se someterá a cada uno y de forma secuencial de mejor a peor una evaluación que definirá si el candidato era realmente un preámbulo. Obviamente, si ya el primero resulta ser un preámbulo, se detiene la sincronización.

Para cada candidato que se quiera evaluar, se calcula el `sync_offset` sumando las tres fuentes de offset mencionadas (sub-chip, chip y símbolo), que ya están asociadas al candidato en sí.

```python
sync_offset = sub_chip_offset + chip_offset + symbol_offset
```
### 3.5 Compensación de CFO y SFO

Durante la recepción, la frecuencia de muestreo del receptor o su frecuencia central pueden estar levemente desalineadas respecto del transmisor. Esto se conoce como **desalineación de frecuencia de muestreo (SFO)** y **desalineación de frecuencia portadora (CFO)**. Estas imperfecciones introducen errores acumulativos en la fase, afectando la demodulación correcta de los símbolos.

Para estimar y corregir estos errores, se utiliza la zona del preámbulo como referencia. Específicamente, se aplica una regresión lineal sobre las correcciones finas (`deltas`) extraídas durante la demodulación para modelar el error como:

$$
\Delta[n] \approx a \cdot n + b
$$

Donde:

* $a$ es el **residuo de SFO**
* $b$ es el **residuo de CFO**

Si cualquiera de estos residuos supera su umbral, se aplica una corrección compleja de fase al buffer sincronizado:

$$
\text{corr}[n] = e^{-j (2\pi \cdot \text{CFO} \cdot \frac{n}{N} + \pi \cdot \text{SFO} \cdot (\frac{n}{N})^2)}
$$

Donde $N$ es la cantidad de chips por símbolo.

```python
def apply_cfo_sfo_correction(self, iq, cfo_residue: float, sfo\_residue: float):
    chips = 1 << self.phy_params.spreading_factor
    sps = chips * self.phy_params.samples_per_chip
    n = np.arange(iq.shape[0], dtype=np.float64)
    cfo_phase = 2.0 * np.pi * cfo_residue * n / chips
    sfo_phase = 2.0 * np.pi * sfo_residue * (n / sps) ** 2 / 2.0
    correction = np.exp(-1j * (cfo_phase + sfo_phase)).astype(iq.dtype, copy=False)
    return iq * correction
```

Los umbrales para aplicar esta corrección son:

```python
CFO_THRESHOLD = 0.1
SFO_THRESHOLD = lambda sf, spc: (1 << sf) / (1000 * spc)
```

La compensación solo se aplica si está habilitada (`compensate_cfo_sfo=True`) **y** los residuos superan dichos umbrales.

```python
if self.compensate_cfo_sfo and should_apply_compensation:
    synced = self._apply_cfo_sfo_correction(synced, cfo, sfo)
```

Esta etapa mejora considerablemente la tasa de detección de tramas válidas cuando existen pequeñas desincronizaciones entre TX y RX, y fue pensada a partir del paper [From Demodulation to Decoding: Toward Complete LoRa PHY Understanding and Implementation](https://dl.acm.org/doi/10.1145/3546869)



Luego, e recorta el buffer a partir de ese punto y se verifican los downchirps del SFD.
### 3.6 Verificación de la zona SFD

Una vez sincronizado tentativamente, se busca el par `(0,0)` de downchirps que define el final de la zona SFD. Si no se encuentra, se descarta el candidato. Para hacer más eficiente la búsqueda, ésta se reduce a solo porciones del buffer en las que tenga sentido que hallan downchirps, es decir, al final de la corrida de candidatos "upchirp". 

```python
# score (0,0) down‑chirp pairs inside mask
pair   = mask[:-1] & mask[1:] & (down_syms[:-1] == 0) & (down_syms[1:] == 0)
```

```python
sfd_end_offset = check_sfd(...)
```

### 3.7 Demodulación del header

A partir del final del SFD, se demodulan dos símbolos para conocer la longitud del payload, que se codifica en el header.

```python
header_syms, _ = demod.demodulate(header, base="downchirp")
payload_len = decode_payload_length(...)
```

### 3.8 Extracción del payload

Se extraen exactamente `payload_len` símbolos (convertidos a muestras) a partir del header. Si se solicitan, también se devuelven offsets estructurales de toda la trama en un diccionario `viz_bundle` para poder visualizarlo en contextos de visualización o debugging.
Por motivos de debugging, también existe la posibilidad de devolver `traces` que almacena información exhaustiva respecto a como fue la sincronización en cada momento, y que errores locales (esperados en situaciones ruidosas) fueron descartando candidatos espurios.

```python
payload_frame = body[frame_samples : frame_samples + needed_samples]
return payload_frame, debug_bundle
```

### 3.9 Robustez

El sistema prueba múltiples candidatos y solo rechaza la sincronización si todos fallan. Esto permite tolerancia ante errores de ruido o desalineación extrema.

A esto se le agrega la parametrización de sus candidatos, para poder elegir entre eficiencia y tolerancia.


## 4. Sincronización por Correlación

Este método utiliza una estrategia tradicional: correlacionar el buffer IQ con un patrón LoRa conocido (el preámbulo más el sync word más el SFD).  
El objetivo es encontrar los puntos del buffer donde la correlación es alta, lo cual sugiere el inicio de una trama.

### 4.1 Qué necesita

- Un patrón de sincronización (`sync_base`) generado a partir de los parámetros del modem.
- Una función de correlación cruzada entre la señal y ese patrón.
- Un umbral de correlación (configurable, por defecto 0.9) para decidir qué puntos del buffer podrían ser válidos.

### 4.2 Qué hace

1. Realiza la correlación cruzada entre `iq_samples` y `sync_base` usando:

   ```python
   y_rev_conj = xp.conj(xp.flip(sync))
   corr = xp.convolve(sig, y_rev_conj, mode='full')
   ```

2. Busca los máximos de correlación que superen el umbral definido:

   ```python
   indices = xp.where(corr > threshold)[0]
   ```
3. Usa el mejor pico como estimación del comienzo de la trama.

4. A partir de ahí, demodula el header, obtiene la longitud del payload y recorta el buffer en consecuencia.

5. Si se pide, devuelve además un diccionario debug_bundle con los offsets clave de la trama.

Este método es más simple que el de dechirping, pero menos robusto en presencia de CFO o tramas mal alineadas, ya que depende de una buena correlación exacta con un patrón predefinido.

